In [ ]:
import os
import time
import random
import hashlib
import requests
import pandas as pd
import urllib.parse

from PIL import Image
from tqdm import tqdm
from io import BytesIO
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Constants
WEBDRIVER_DELAY_TIME_INT = 20
TIMEOUT_INT = 20
CHROMEDRIVER_PATH = '/usr/bin/chromedriver'


def get_chrome_options():
    """ Set up Chrome options for Selenium WebDriver."""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Enables headless mode
    # Avoid sandbox issues in some environments
    options.add_argument("--no-sandbox")
    # Handles insufficient /dev/shm space
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("window-size=1920x1080")  # Set window size
    return options


def initialize_driver():
    """ Initialize the Selenium WebDriver with Chrome options."""
    service = Service(executable_path=CHROMEDRIVER_PATH)
    chrome_options = get_chrome_options()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.implicitly_wait(TIMEOUT_INT)  # Implicit wait for elements
    return driver


# Example usage
driver = initialize_driver()
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)  # Explicit wait

In [ ]:
def get_image_links_from_page(page_url, driver):
    driver.get(page_url)
    try:
        container = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div.FS5UE28h.container")
        ))
        image_items = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "div.LQY5mtmC div.aLnnpRah.text-center"))
        )

        image_links = []
        for img_elem in image_items:
            img_div = img_elem.find_element(
                By.CSS_SELECTOR, "div.Mw1EAtrx img, img")

            img_url = img_div.get_attribute("src")
            img_title = img_div.get_attribute("title")
            if img_url:
                image_links.append((img_url, img_title))

        return image_links
    except Exception as e:
        print(f"Error while trying to extract images: {e}")
        return []


def hash_image_content(url):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            return hashlib.md5(response.content).hexdigest()
        else:
            print(
                f"Error downloading image from {url}; status: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error with the image download for {url}: {e}")
        return None


def convert_webp_to_jpg(webp_data):
    try:
        img = Image.open(BytesIO(webp_data))
        if img.format == 'WEBP':
            if img.mode == 'RGBA':
                img = img.convert('RGB')
            buffer = BytesIO()
            img.save(buffer, format="JPEG")
            return buffer.getvalue()
        else:
            return webp_data
    except Exception as e:
        print(f"Error converting WebP to JPG: {e}")
        return webp_data


def download_image(img_url, img_name, folder_path):
    try:
        response = requests.get(img_url, stream=True)
        if response.status_code == 200:
            img_path = os.path.join(folder_path, f"{img_name}")
            img_data = convert_webp_to_jpg(response.content)
            with open(img_path, "wb") as f:
                f.write(img_data)
        else:
            print(
                f"Error downloading image from {img_url}; status: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error with the image download for {img_url}: {e}")


def process_image_page(image_url, img_title, folder_path, idx, tag, seen_hashes):
    img_hash = hash_image_content(image_url)
    if img_hash and img_hash not in seen_hashes:
        seen_hashes.add(img_hash)
        new_file_name = f"{tag}_{idx:07d}.jpg"
        download_image(image_url, new_file_name, folder_path)
        metadata = {
            "file_name": new_file_name,
            "image_url": image_url,
            "image_title": img_title,
            "tag": tag
        }
        return metadata
    else:
        return None


def loop_over_pages(base_url, tags, total_pages, driver, folder_path):
    os.makedirs(folder_path, exist_ok=True)
    all_metadata = []
    seen_hashes = set()

    for tag in tags:
        all_images = []

        for page in tqdm(range(1, total_pages + 1), desc=f"Extracting Images for {tag}", unit="page"):
            page_url = f"{base_url}/emoji-list/tag/{tag}?page={page}"
            images = get_image_links_from_page(page_url, driver)
            all_images.extend(images)

            time.sleep(1)

        metadata_list = []
        for idx, (img_url, img_title) in enumerate(all_images, start=1):
            metadata = process_image_page(
                img_url, img_title, folder_path, idx, tag, seen_hashes)
            if metadata:
                metadata_list.append(metadata)

        all_metadata.extend(metadata_list)

    return all_metadata


def save_metadata(metadata_list, metadata_file):
    df = pd.DataFrame(metadata_list)
    df.to_csv(metadata_file, index=False, encoding="utf-8")

In [ ]:
os.makedirs("crawled_data", exist_ok=True)
folder_path = os.path.join("crawled_data", "images")
metadata_file = os.path.join("crawled_data", "metadata.csv")

base_url = "https://discords.com"
tags = ["Panda"]
total_pages = 1000

metadata_list = loop_over_pages(
    base_url, tags, total_pages, driver, folder_path)
save_metadata(metadata_list, metadata_file)

print("Start downloading images...")
with tqdm(total=len(metadata_list), desc="Downloading Images", unit="image") as pbar:
    for metadata in metadata_list:
        img_url = metadata['image_url']
        file_name = metadata['file_name']
        download_image(img_url, file_name, folder_path)
        pbar.update(1)

print("Download images completed.")

total_crawled_images = len(os.listdir(folder_path))
print(f"Total crawled images: {total_crawled_images}.")

driver.quit()